### Before 2014 and After 2014

In [268]:
#parsing pre files
from PyPDF4 import PdfFileReader
import numpy as np
import pandas as pd
import os
import re


In [265]:
path = ".\\ABHI\\"
ind_pos = [0,18,19,20,38,39, 42,41,43,45,49,50,53,54,53,61,66,68,69,76,82,83,84,91,96,102]
#ack number , member id, mem name, mobile  email
pdffiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.pdf')]
rows=[]
def process_post(cleaned):
    regex = r'\d{2}\-[A-Z]{3}\-\d{4}'
    if not re.fullmatch(regex, cleaned[20]):
        cleaned[19] = cleaned[19] + cleaned[20]
        cleaned.pop(20)
    
    if not len(cleaned[39]) == 15:
        cleaned[39] = cleaned[39] + cleaned[40]

    if cleaned[46] != '9':
        cleaned[45] = cleaned[45] + cleaned[46]
        cleaned.pop(46)

    if cleaned[51] != '5':
        cleaned[50] = cleaned[50] + cleaned[51]
        cleaned.pop(51)

    if not cleaned[77].startswith('Whether Employer'):
        cleaned[76]=cleaned[76]+cleaned[77]
        cleaned.pop(77)
        
    return [cleaned[0],cleaned[39],cleaned[19],cleaned[69],cleaned[76], fname,"post 2014"]
    return False

def process_pre(cleaned):
    if not len(cleaned[49]) == 15:
        cleaned[49] = cleaned[49] + cleaned[50]
        
    if cleaned[87] != '14':
        cleaned[86] = cleaned[86] + cleaned[87]
        cleaned.pop(87)
    # print(cleaned[95:107])
    return [cleaned[0],cleaned[49],cleaned[37],cleaned[104],cleaned[107], fname,"pre 2014"]
    return False


def process_page(page1, fname):
    cleaned = [string for string in page1.split('\n') if string.strip()]
    if cleaned[13]=='PPO Number':
        arr = process_pre(cleaned)
        return arr
    elif cleaned[13] == 'UAN':
        arr = process_post(cleaned)
        return arr
    else:
        arr = False

    return arr

    

for fname in pdffiles: #['6682_841.pdf','6682_1542.pdf','6682_839 COPY.pdf','10521_28.pdf']:#
    fullpath = path+fname
    pdf_file = open(fullpath, 'rb')
    pdf_reader = PdfFileReader(pdf_file)
    page1 = pdf_reader.getPage(0).extractText()
    pdf_file.close()
    clean = process_page(page1, fname)
    if clean:
        rows.append(clean)

cols=['ack','memid','name','phone','email','fname','category']
df = pd.DataFrame(np.array(rows),columns=cols)
df.to_excel('all.xlsx', index=False)
